# Scrapping Data dari komentar video Youtube 

## Install Selenium
Selenium dapat digunakan untuk membuka browser web, menavigasi ke halaman video yang ingin diambil komentarnya, dan mengambil data komentar dari sumber kode halaman web

In [2]:
! pip install selenium --quiet

## Install NLTK (Natural Language Toolkit)
NLTK digunakan untuk melakukan pemrosesan teks seperti tokenisasi untuk memisahkan teks menjadi kata-kata, stemming untuk mengubah kata-kata ke bentuk dasarnya

In [3]:
! pip install nltk

## Install IndoNLP
IndoNLP digunakan untuk melakukan analisis bahasa pada komentar

In [4]:
! pip install indoNLP

In [16]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrapping Data komentar Youtube

### Mengambil komentar youtube

In [25]:
# Menyimpan data yang akan di crawling
data = []

url = "https://www.youtube.com/watch?v=dmZsg9HP1Yw"

# Menghubungkan driver local ke link youtube yang dituju
with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:    
    wait = WebDriverWait(driver, 10)
    driver.get(
        url)

    # melakukan perulangan untuk scrolling data sebanyak 10 kali
    for item in range(10):
        wait.until(EC.visibility_of_element_located(
            (By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    # mencari elemen dengan tag content pada halaman youtube
    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\HP\AppData\Local\Temp\ipykernel_13340\2733254033.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:


### Mengubah data ke DataFrame

In [26]:
import pandas as pd   
df = pd.DataFrame(data, columns=['komentar'])
df

,komentar
0,"ID\nSkip navigation\nSign in\nPilpres 2024, PD..."
1,
2,Bersatulah untuk orang pemimpin yang Iklas.......
3,Dari ucapan yang disampaikan Pak GP dan PDIP k...
4,Kami sebagai rayat jika sudah ada kepastian ca...
...,...
124,
125,
126,
127,


### Cleaning Data

#### Menghilangkan teks yang tidak digunakan

In [19]:
import re
import string
import numpy as np

# Teks Cleaning


def cleaning(text):
    # Menghilangkan tag bawaan HTML

    text = re.compile(
        '<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # mengubah defaukt teks ke huruf kecil
    text = text.lower()

    # menghapus karakter kosong di depan dan belakang teks
    text = text.strip()

    # menghapus tanda baca, karakter spesial, dan spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # menghapus teks yang berisi angka
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    # Menghapus bawaan teks youtube
    text = re.sub('id', '', text)
    text = re.sub('skip', '', text)
    text = re.sub('navigation', '', text)
    text = re.sub('sign', '', text)

    return text

#### Steaming Data

In [20]:
# Import 
from nltk.stem import PorterStemmer
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang,
                emoji_to_words, remove_html])
# steamming
ps = PorterStemmer()

#### Ubah data awal dan cleaning ke DataFrame

In [21]:
df['hasil komentar'] = df['komentar'].apply(lambda x: pipe(x))
df['hasil komentar'] = df['hasil komentar'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['hasil komentar'] = df['hasil komentar'].replace('', np.nan)

#### Cek nilai yang kosong (NaN)

In [22]:
print(df.isna().sum())

komentar          0
hasil komentar    8
dtype: int64


#### Menghapus data Missing Value (Nan)

In [23]:
df.dropna(inplace=True)

#### Print hasil data

In [24]:
df

,komentar,hasil komentar
0,"ID\nSkip navigation\nSign in\nPilpres 2024, PD...",ini pilpres pdi p megawati umumkan capres ...
2,Bersatulah untuk orang pemimpin yang Iklas.......,bersatulah untuk orang pemimpin yang iklas buk...
3,Dari ucapan yang disampaikan Pak GP dan PDIP k...,dari ucapan yang disampaikan pak gp dan pdip k...
4,Kami sebagai rayat jika sudah ada kepastian ca...,kami sebagai rayat jika sudah ada kepastian ca...
5,Mantap inilah yg di tunggu tunggu Olh rakyat ...,mantap inilah yang di tunggu tunggu olh rakyat...
6,Sebaiknya Ganjar berduet dgn Mahfud..maka pemb...,sebaiknya ganjar berduet dengan mahfud maka pe...
7,Bismillah semoga koalisi PKS Demokrat dan Nasd...,bismillah semoga koalisi pks demokrat dan nasd...
8,Kami hanya mendukung partai politik yang konsi...,kami hanya mendukung partai politik yang konsi...
9,KB ini bisa saja mengaplikasikan cita2 luhur p...,kb ini bisa saja mengaplikasikan cita cita luh...
10,Harapan saya hanya satu.. Yaitu pasangan yang ...,harapan saya hanya satu yaitu pasangan yang ea...


#### Import data ke CSV

In [32]:
# to csv
df.to_csv('Data_KomentarYoutube.csv', index=False)